## 임베딩 사용

이 노트북에는 OpenAI API를 통해 'text-embedding-ada-002' 모델로 텍스트를 임베드하는 데 사용할 수 있는 몇 가지 유용한 스니펫이 포함되어 있습니다.

In [1]:
import openai

embedding = openai.Embedding.create(
    input="Your text goes here", model="text-embedding-ada-002"
)["data"][0]["embedding"]
len(embedding)


1536

API를 너무 빨리 누르면 속도 제한이 트리거될 수 있으므로 API 속도 제한을 더 잘 관리하기 위해 'tenacity' 패키지 또는 다른 지수 백오프 구현을 사용하는 것이 좋습니다. 다음 함수를 사용하면 임베딩을 최대한 빠르게 가져올 수 있습니다.

In [ ]:
# Negative example (slow and rate-limited)
import openai

num_embeddings = 10000 # Some large number
for i in range(num_embeddings):
    embedding = openai.Embedding.create(
        input="Your text goes here", model="text-embedding-ada-002"
    )["data"][0]["embedding"]
    print(len(embedding))

In [2]:
# Best practice
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt

# Retry up to 6 times with exponential backoff, starting at 1 second and maxing out at 20 seconds delay
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_embedding(text: str, model="text-embedding-ada-002") -> list[float]:
    return openai.Embedding.create(input=[text], model=model)["data"][0]["embedding"]

embedding = get_embedding("Your text goes here", model="text-embedding-ada-002")
print(len(embedding))

1536
